In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import warnings
from nltk.corpus import stopwords
warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline

#https://na80.lightning.force.com/lightning/r/Report/00O1P000003mYjbUAE/view

/Users/emeryames/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [29]:
df = pd.read_csv('year_data.csv')
df.Description.head()

0    Chat transcript\n\n\n\t\n\t\t\n\t\t\tName: GAr...
1    Chat transcript\n\n\n\t\n\t\t\n\t\t\tName: Gen...
2    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: c...
3    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: c...
4    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: e...
Name: Description, dtype: object

In [31]:
data = df.Description.values.tolist()
data = [re.sub('\S*@\S*\s?', "", sent) for sent in data]
data = [re.sub('\s+', " ", sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub('https:\S+',"", sent) for sent in data]
data = [re.sub('Chat transcript',"", sent) for sent in data]
data = [re.sub('BevSpot Support here! How can we help',"", sent) for sent in data]
data = [re.sub('Chat has been opened from',"", sent) for sent in data]
data = [re.sub('E-mail', "", sent) for sent in data]
data = [re.sub('..:..:.. ..', "", sent) for sent in data]
data = [re.sub('America\S+', "", sent) for sent in data]
data = [re.sub('\S+.com\S+', "", sent) for sent in data]
data = [re.sub('BevSpot', "", sent) for sent in data]

pprint(data[:1])

[' Name: GAry Hipko : Brad Mon, 01/04/21   Hi from  Support! Check out our '
 'help center at  or chat us here if you have any questions as you move around '
 'the software. GAry Hipko  my food account is locked. why? Brad  Hi, Gary let '
 'me look into this for you. Gary, I am seeing a delinquency on the food '
 'account. Your grace period ended on 12/10. I am going to see if I can talk '
 'to your account manager. Do you mind waiting a few minutes? GAry Hipko  ive '
 'been waiting 1 1/2 hours for a call back. whats a few more minutes... Brad  '
 'Gary, I am very sorry to hear that. Did you have an appointment already '
 'scheduled? GAry Hipko  no i left a message Brad  Okay. Let me see what I can '
 'do. Gary, are you still with me? GAry Hipko  yes Brad  I am going to send '
 'you a link to Susans calendar so you can schedule an appointment to speak '
 'with her. I am also going to temporarily unlock the account until you '
 'resolve the billing issue with her. GAry Hipko  thanks Bra

In [32]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data))

In [131]:
bigram = gensim.models.Phrases(data, min_count=10, threshold=100)
trigram = gensim.models.Phrases(bigram[data], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [211]:
stop_words = stopwords.words('english')
stop_words.extend (('chat','let','go','see','thank','get','done','email','take','look','hang','tight','show','reach','send','thanks',
                    'im','one','help','went','said','hi','gm','way','best','need','one','got','much','want','check','youll','sure',
                    'hello','hear','us','name','ok','okay',
                    'erick','victoria','marguerite'))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [212]:
data_words_nostops = remove_stopwords(data)
data_words_bigrams = make_bigrams(data_words_nostops)

In [213]:
print(data_words_bigrams)

[['gary', 'hipko', 'brad', 'mon', 'support', 'center', 'questions', 'move', 'around', 'software', 'gary', 'hipko', 'food', 'account', 'locked', 'brad', 'gary', 'gary', 'seeing', 'delinquency', 'food', 'account', 'grace', 'period', 'ended', 'going', 'talk', 'account', 'manager', 'mind', 'waiting', 'minutes', 'gary', 'hipko', 'ive', 'waiting', 'hours', 'call', 'back', 'whats', 'minutes', 'brad', 'gary', 'sorry', 'appointment', 'already', 'scheduled', 'gary', 'hipko', 'left', 'message', 'brad', 'gary', 'still', 'gary', 'hipko', 'yes', 'brad', 'going', 'link', 'susans', 'calendar', 'schedule', 'appointment', 'speak', 'also', 'going', 'temporarily', 'unlock', 'account', 'resolve', 'billing', 'issue', 'gary', 'hipko', 'brad', 'gary', 'access', 'account', 'gary', 'hipko', 'without', 'leaving', 'brad', 'please', 'copy', 'link', 'schedule', 'appointment', 'susan', 'gary', 'hipko', 'brad', 'great', 'day', 'gary', 'gary', 'hipko', 'toto', 'would', 'unexpected', 'token', 'json', 'position', 'thism

In [214]:
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [215]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

In [224]:
#tweak topic for best score
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=13, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=25,
                                            minimum_probability=.8,
                                           alpha=0.01,
                                           per_word_topics=True)
#score
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.303313899753115

Coherence Score:  0.33764680539422537


In [225]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.273188 -0.172122       1        1  26.226122
3     -0.326541 -0.013144       2        1  21.172697
2     -0.257723  0.111449       3        1  13.006847
1      0.092252 -0.340285       4        1   7.397931
5     -0.091270 -0.261961       5        1   7.043658
7      0.147131  0.316026       6        1   6.343289
0     -0.140886  0.173090       7        1   5.303789
10    -0.043319  0.279535       8        1   4.356597
4      0.036129 -0.091893       9        1   3.838043
8      0.288415 -0.150521      10        1   3.521352
12     0.250898  0.029950      11        1   0.758910
11     0.182746  0.090390      12        1   0.614694
6      0.135355  0.029485      13        1   0.416071, topic_info=            Term        Freq       Total Category  logprob  loglift
1384      kendra  176.000000  176.000000  Default  30.0000  30.0000
327     nicholas   89.000000   89.000000  Default  29.0000  29.0000
2        account  305.000000  305.000000  Default  28.0000  28.0000
321        anton  130.000000  130.000000  Default  27.0000  27.0000
646      senneca   71.000000   71.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
1567  finalizing    0.023698    0.845887  Topic13  -8.1713   1.9071
1896    castillo    0.023653    0.830480  Topic13  -8.1732   1.9236
1898         ftl    0.023653    0.830480  Topic13  -8.1732   1.9236
1899       green    0.023653    0.830480  Topic13  -8.1732   1.9236
1901       typed    0.023653    0.830480  Topic13  -8.1732   1.9236

[693 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.261039     able
0         2  0.408583     able
0         3  0.158894     able
0         5  0.045398     able
0         8  0.124845     able
...     ...       ...      ...
118       7  0.037955      yes
118       9  0.022773      yes
187       1  0.903353      yet
877       5  0.907752  zachary
1118      5  0.989741     zeke

[1040 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 4, 3, 2, 6, 8, 1, 11, 5, 9, 13, 12, 7])